In [1]:
import numpy as np
import os
import datetime
import subprocess
import shutil

white = True
erase = False
rows = 20
cols = rows
dim = 800
radius = dim/(cols)
g = False
grid = np.zeros((rows, cols))
record = False
saving = False
frame_counter = 0
timestamp = ""

os.makedirs('saves', exist_ok=True)

def setup():
    size(dim, dim)
    no_loop()
    stroke_weight(3)

def draw():
    global grid, white, radius, g, saving, record, frame_counter

    if white:
        background(0)
        fill(255)
        stroke(255)
    else:
        background(255)
        fill(0)
        stroke(0)

    w_c = width / grid.shape[1]
    h_c = height / grid.shape[0]

    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            x = (j + 0.5) * w_c
            y = (i + 0.5) * h_c
            if grid[i][j] == 1:
                circle(x, y, radius)

    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            x1 = (j + 0.5) * w_c
            y1 = (i + 0.5) * h_c
            if grid[i][j] == 1:
                neighbours = [(i, j + 1), (i + 1, j), (i + 1, j - 1), (i + 1, j + 1)]
                for i2, j2 in neighbours:
                    if i2 in range(0, grid.shape[0]) and j2 in range(0, grid.shape[1]):
                        if grid[i2][j2] == 1:
                            x2 = (j2 + 0.5) * w_c
                            y2 = (i2 + 0.5) * h_c
                            if abs((i2 + j2) - (i + j)) == 1:
                                rect_c(np.mean([x1, x2]), np.mean([y1, y2]), w_c)
                            else:
                                conj(np.mean([x1, x2]), np.mean([y1, y2]), radius / 2)

    if g:
        for i in range(grid.shape[0]):
            stroke(255, 0, 0, 180)
            stroke_weight(2)
            line(0, i * w_c, width, i * w_c)
            line(i * w_c, 0, i * w_c, height)
            stroke_weight(3)

    if saving:
        time_frame = datetime.datetime.now().strftime('%y%m%d%H%M%S')
        save_frame(f'saves/{time_frame}.png')
        saving = False

    if record:
        save_frame(f'saves/{timestamp}/{frame_counter}.png')
        frame_counter += 1

def mouse_dragged():
    global grid, erase, radius
    x_i = int(mouse_x // radius)
    y_i = int(mouse_y // radius)
    if erase:
        grid[y_i][x_i] = 0
    else:
        grid[y_i][x_i] = 1
    redraw()

def mouse_pressed():
    global grid, erase, radius
    x_i = int(mouse_x // radius)
    y_i = int(mouse_y // radius)
    if erase:
        grid[y_i][x_i] = 0
    else:
        grid[y_i][x_i] = 1
    redraw()

def key_pressed():
    global white, erase, g, saving, record, grid, timestamp, frame_counter

    if key == 'w':
        white = not white
        redraw()
    elif key == 'm':
        erase = not erase
    elif key == 'g':
        g = not g
        redraw()
    elif key == 'r':
        if not record:
            # nuova registrazione
            timestamp = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
            frame_counter = 0
            os.makedirs(f'saves/{timestamp}', exist_ok=True)
            record = True
            loop()
        else:
            # fine registrazione
            record = False
            no_loop()
            redraw()
            println("Creazione video")
            make_video_from_frames(timestamp)
            println("Video salvato")
    elif key == 's':
        saving = True
        redraw()
    elif key == 'x':
        grid = np.zeros((rows, cols))
        redraw()

def rect_c(x, y, size):
    rect_mode(CENTER)
    square(x, y, size)

def conj(xc, yc, radius):
    k = 4 / 3 * (sqrt(2) - 1)
    begin_shape()
    vertex(xc, yc - radius)
    bezier_vertex(xc, yc - radius + k * radius,
                  xc + radius - k * radius, yc,
                  xc + radius, yc)
    bezier_vertex(xc + radius - k * radius, yc,
                  xc, yc + radius - k * radius,
                  xc, yc + radius)
    bezier_vertex(xc, yc + radius - k * radius,
                  xc - radius + k * radius, yc,
                  xc - radius, yc)
    bezier_vertex(xc - radius + k * radius, yc,
                  xc, yc - radius + k * radius,
                  xc, yc - radius)
    end_shape(CLOSE)

def make_video_from_frames(folder):
    input_dir = f'saves/{folder}'
    output_video = f'saves/{folder}.mp4'
    input_pattern = f'{input_dir}/%d.png'

    command = [
        'ffmpeg',
        '-y',
        '-framerate', '30',
        '-i', input_pattern,
        '-c:v', 'libx264',
        '-pix_fmt', 'yuv420p',
        output_video
    ]

    try:
        subprocess.run(command, check=True)
        
    except subprocess.CalledProcessError as e:
        println(f"Errore nella creazione del video: {e}")
        println(f"Codice di uscita: {e.returncode}")
        # Non eliminare la cartella in caso di errore, così rimane per il debug
    shutil.rmtree(input_dir)
run_sketch()


Creazione video
Video salvato
